In [1]:
from ema_workbench import Scenario, Policy, MultiprocessingEvaluator, ema_logging, load_results
from ema_workbench.analysis import prim
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.em_framework.evaluators import BaseEvaluator
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ema_workbench.analysis import parcoords
import seaborn as sns
import funs_project as fp

# Optimisation

In [2]:
ACTORNAME = "Deventer"

In [ ]:
dike_model, planning_steps = get_model_for_problem_formulation(ACTORNAME)
outcomekeys = [outcome.name for outcome in dike_model.outcomes]

## Reading
The results from the optimization. The policies that are selected with the cure_policy_selection.

In [ ]:
cases = fp.the_cases(ACTORNAME)

In [ ]:
policies = fp.crude_policy_selection(ACTORNAME, 6)

In [ ]:
policies = policies.sort_values(by="scenario")
policies

In [ ]:
#policies.iloc[:, 0:10]

Below we plot per scenario 

In [ ]:
colors = iter(sns.color_palette())
limits = parcoords.get_limits(policies.iloc[:, 10:13])

# limits.loc[0, ['inertia', 'reliability']] = 1
# limits.loc[0, 'max_P'] = 4 # max over results based on quick inspection not shown here
# limits.loc[0, 'utility'] = 1 # max over results based on quick inspection not shown here
# limits.loc[1, :] = 0
paraxes = parcoords.ParallelAxes(limits)


for index, row in policies.iterrows():
    color = next(colors)
    paraxes.plot(row, label=f'scenario {cases[row.scenario]}', color=color)

paraxes.legend()
plt.rcParams["figure.figsize"] = (10,10)
#plt.tight_layout()
plt.show()


# Reevaluate under deep uncertainty

## Reading
Read in the results from the reevaluate under deep uncertainty


In [ ]:
n_scenarios = 1000

In [ ]:
du_experiments, du_outcomes = load_results("simulation/optimisation/du_scen_" + str(n_scenarios) + "_" + ACTORNAME + ".tar.gz")

### Keep only the DU experiments and outcomes that are in POLICIES
This way only the selected policies (and the respective scenarios) are brought into the rest of the analysis.

In [ ]:
du_outcomes_df = pd.DataFrame.from_dict(du_outcomes)
merged_du = pd.concat([du_experiments, du_outcomes_df], axis=1)
merged_du.head(5)

In [ ]:
#now compare the policies in merged_du to the policies in 'policies' 
policies_policy_df = policies.iloc[:, 0:10]
DU_policy_selected_df = pd.merge(policies_policy_df, merged_du, how = "inner")
DU_policy_selected_df.head(5)

#### rewrite du_experiments and du_outcomes with the filtered version

In [ ]:
du_outcomes_policy_df = DU_policy_selected_df[DU_policy_selected_df.columns[-3:]].copy()
du_experiments = DU_policy_selected_df[DU_policy_selected_df.columns[0:50]].copy()
du_outcomes = du_outcomes_policy_df.to_dict('series')

## Regret
Comparing alternatives
"With a regret view, a decision maker wants to minimize the regret of choosing incorrectly, where regret is the loss in performance. This regret could be the cost of assuming the wrong future scenario or the cost of choosing the wrong alternative. In the first case, maximum regret of an alternative is the difference between its performances in the best estimate future scenario and in the scenario where it shows the worst performance, for example, as applied by Kasprzyk et al. (2013). In the second case, the regret of an alternative in a certain future scenario is the difference between its performance and the best-performing alternative, as proposed by Savage (1951). Maximum regret of an alternative is its highest regret achieved over all future scenarios. In both regret cases, the alternative with the smallest maximum regret is the most robust."

https://link.springer.com/article/10.1007/s13595-017-0641-2

In [ ]:
outcomekeys = list(du_outcomes.keys())
outcomekeys

In [ ]:
DAMAGE = outcomekeys[0]
DEATHS = outcomekeys[1]
COSTS = outcomekeys[2]

#### Regret calculation based on assignments

In [ ]:
overall_scores = {}
regret = []
for scenario in du_experiments.scenario.unique():
    logical = du_experiments.scenario==scenario
    temp_results = {k:v[logical] for k,v in du_outcomes.items()}
    temp_results = pd.DataFrame(temp_results)
    temp_experiments = du_experiments[du_experiments.scenario==scenario]
    best = temp_results.min() #we are minimizing
    scenario_regret = temp_results - best
    scenario_regret['policy'] = temp_experiments.policy.values
    regret.append(scenario_regret)

In [ ]:
regret = pd.concat(regret)
maxregret = regret.groupby('policy').max().dropna()#dropna to remove some artifacts that were left

In [ ]:
maxregret

### renaming policies for legend and colourssss
rename policy names to things that will stick and we'll remember
In the order that they are above, replace with 
D_0, D_1, ...  D_10, D_11,

At the same time create a dictionary for the colours that is linked to the policy name, so that for the following graphs, the same policies have the same colour.

In [ ]:
#naming and colour block
old_names = maxregret.index.values.tolist()
new_names = []
dict_naming = {}

for i in range(len(maxregret)):
    name = "D_" + str(i)
    #print(name)
    new_names.append(name)
dict_naming = {old_names[i]: new_names[i] for i in range(len(old_names))}
#dict_naming  

#Dictionary for the coloursss
color_list =  sns.color_palette('Spectral',len(old_names))
dict_colours = {old_names[i]: color_list[i] for i in range(len(old_names))}


In [ ]:
from matplotlib import cm

limits = parcoords.get_limits(maxregret)
paraxes = parcoords.ParallelAxes(maxregret)


for index, row in maxregret.iterrows():
    paraxes.plot(row, color=dict_colours[index], label=dict_naming[index])
    
paraxes.legend()
plt.figtext(.5,1,'Regret results - Deventer',fontsize=25,ha='center')

plt.savefig("../../report/figures/results/regret_figure"+ "_" + ACTORNAME+ ".png", bbox_inches="tight")
plt.show()

## Satisficing
Performance threshold
"seeks a decision alternative that meets his or her performance requirements over the range of plausible future scenarios. "

For the satisficing analysis, the domain criterion metric from the assignments and as discussed in the paper: [(Bartholomew, Kwakkel 2020)](https://repository.tudelft.nl/islandora/object/uuid%3A17668d72-4ae4-47a4-9905-ebb0e1e75128). Another option that we explored was looking over the policies and seeing if there are any policies that are within all three thresholds. 

The threshold values are found within funs_project.py and sources can be found within the acompanying report.

In [ ]:
satisfycing_df = policies.copy()
satisfycing_df.drop_duplicates(inplace=True)
#satisfycing_df

In [ ]:
belowtresh_df = satisfycing_df.loc[(satisfycing_df['Deventer Expected Annual Damage'] < fp.thresholds_deventer['Deventer Expected Annual Damage']) & \
                    (satisfycing_df['Deventer Expected Number of Deaths'] < fp.thresholds_deventer['Deventer Expected Number of Deaths']) & \
                    (satisfycing_df['Deventer Total Costs'] < fp.thresholds_deventer['Deventer Total Costs']),:].copy()
belowtresh_df.sort_index(inplace=True, ascending = False)
belowtresh_df

From the 12 selected policies there are two policies that fall completly within all the thresholds for Deventer. 

### Domain-criterion analysis

In [ ]:
limits_df = belowtresh_df.iloc[:, -4:-1]
colors = iter(sns.color_palette())
limits = parcoords.get_limits(limits_df)

In [ ]:

overall_scores = {}
for policy in du_experiments.policy.unique():
    logical = du_experiments.policy == policy
    scores = {}
    for k, v in du_outcomes.items():
        try:
            n = np.sum(v[logical]<=fp.thresholds_deventer[k])#check if it is below thresholds
        except KeyError:
            continue
        scores[k] = n/1000 
    overall_scores[policy] = scores
        
overall_scores = pd.DataFrame(overall_scores).T

In [ ]:
from matplotlib import cm

limits = parcoords.get_limits(overall_scores)
paraxes = parcoords.ParallelAxes(limits)

for index, row in overall_scores.iterrows():
    paraxes.plot(row, color=dict_colours[index], label=dict_naming[index])
    
paraxes.legend()
plt.figtext(.5,1,'Satisficing results - Deventer',fontsize=25,ha='center')

plt.savefig("../../report/figures/results/domain_criterion"+ "_" + ACTORNAME+ ".png", bbox_inches="tight")
plt.show()

In [ ]:
overall_scores

## Scoring policies
This part combines the results from both robustness measures to score the policies and select the most robust 5 policies. The regret results are normalised first to make it possible to get a 'good' average. This will be done by first taking the average of the two metrics and than sorting first by the domain-criterion and then by regret.

### Normalise the regret and take the average to score the policies


In [ ]:
from sklearn import preprocessing

In [ ]:
#Normalize the regret results
regret_scores = maxregret.copy()
regret_average = regret_scores.apply(lambda x: x/x.max(), axis=0)

In [ ]:
#Get the average for each policy
regret_average['average regret'] = regret_average.mean(numeric_only=True, axis=1)
regret_average.sort_values(by='average regret', ascending = False, inplace = True) #Lower = better
regret_average

###  Use the satisficing / domain criterion and take their average to score the policies


In [ ]:
#Get the average for each policy
satisficing_average = overall_scores.copy()
satisficing_average['average satisficing'] = satisficing_average.mean(numeric_only=True, axis=1)
satisficing_average.sort_values(by='average satisficing', ascending = True, inplace = True) #higher = better
satisficing_average

### Add them together  

In [ ]:
merged_df = pd.concat([satisficing_average, regret_average], join="inner", axis = 1)


In [ ]:
merged_df.sort_values(by='average satisficing', inplace = True, ascending = True)
merged_df.sort_values(by='average regret', inplace = True, ascending = False)
merged_df

In [ ]:
#take te most robust policies and put them into a df.
robust_policies = merged_df.tail(5)
robust_policies

The policies above show that there often is a trade-off between satisficing and regret. 

## Tying it back to the policy levers.

In [ ]:
#now to return to an original list of policies with this
policy_names = robust_policies.index.values.tolist()

In [ ]:
temp_results = DU_policy_selected_df[DU_policy_selected_df['policy'].isin(policy_names)]

In [ ]:
lever_names = policies_policy_df.columns.values.tolist()


In [ ]:
robust_policies_results = temp_results[lever_names].drop_duplicates()

In [ ]:
robust_policies_results

In [ ]:
robust_policies_results.to_csv('simulation/selected/selected_policies_' + ACTORNAME + '.csv')

In [ ]:
lever_names.append('policy')
robust_policies_results_names = temp_results[lever_names].drop_duplicates()
robust_policies_results_names.set_index('policy', inplace = True)
robust_policies_results_names

In [ ]:
robust_policies_results_names["new_policy_name"] = np.nan
for index, row in robust_policies_results_names.iterrows():
    robust_policies_results_names.at[index, "new_policy_name"] = dict_naming[index]
robust_policies_results_names.to_csv('simulation/selected/selected_policies_NAMES_' + ACTORNAME + '.csv')
robust_policies_results_names